<a href="https://colab.research.google.com/github/mrudula77/House_Price_Predictions/blob/main/HousePricePredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing libraries and dataset**

In [250]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [251]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import seaborn as sns

In [252]:
train_data = pd.read_csv('/content/drive/MyDrive/Datasets/HousePricePrediction/train (1).csv')

#**Data exploration and preprocessing**

In [253]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [254]:
train_data.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [255]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [256]:
train_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [257]:
train_data.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

#**Feature Engineering and Null values handling**

In [258]:
y=train_data.SalePrice
print(y)

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64


In [259]:
# features=['LotArea','YearBuilt','1stFlrSF','2ndFlrSF','FullBath','BedroomAbvGr','TotRmsAbvGrd']
features=['MSSubClass'
          ,'LotArea'
          ,'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
          'MasVnrArea',
          'BsmtFinSF1',
          'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
          '1stFlrSF', '2ndFlrSF',
          'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
          'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
          'TotRmsAbvGrd',
         'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
        'MiscVal',  'MoSold', 'YrSold']
X= train_data[features]
print(X.head())

   MSSubClass  LotArea  OverallQual  OverallCond  YearBuilt  YearRemodAdd  \
0          60     8450            7            5       2003          2003   
1          20     9600            6            8       1976          1976   
2          60    11250            7            5       2001          2002   
3          70     9550            7            5       1915          1970   
4          60    14260            8            5       2000          2000   

   MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  ...  GarageArea  WoodDeckSF  \
0       196.0         706           0        150  ...         548           0   
1         0.0         978           0        284  ...         460         298   
2       162.0         486           0        434  ...         608           0   
3         0.0         216           0        540  ...         642           0   
4       350.0         655           0        490  ...         836         192   

   OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPo

In [260]:
X=X.fillna(-1)

#**Model Building**

In [261]:
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1)

In [262]:
rf_model= RandomForestRegressor(random_state=1, n_estimators=700)
rf_model.fit(train_X,train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)
rf_val_rmse = np.sqrt(mean_squared_error(rf_val_predictions, val_y))

print('Validation MAE for Random Forest Model: {:,.0f}'.format(rf_val_mae))
print(f'Validation RMSE for Random Forest Model: {rf_val_rmse}')

Validation MAE for Random Forest Model: 16,559
Validation RMSE for Random Forest Model: 25547.61859446015


In [263]:
rf_model_on_full_data = RandomForestRegressor(random_state=1, n_estimators=700)
rf_model_on_full_data.fit(X,y)

RandomForestRegressor(n_estimators=700, random_state=1)

#**Model Testing and Predictions**

In [264]:
test_data = pd.read_csv('/content/drive/MyDrive/Datasets/HousePricePrediction/test (1).csv')
test_data = test_data.fillna(-1)

In [265]:
test_X = test_data[features]
test_preds = rf_model_on_full_data.predict(test_X)
print(test_preds)

[128588.23714286 155344.22857143 180664.79571429 ... 159784.51428571
 112860.23       232659.54428571]


In [266]:
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,-1,Reg,Lvl,AllPub,...,120,0,-1,MnPrv,-1,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,-1,IR1,Lvl,AllPub,...,0,0,-1,-1,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,-1,IR1,Lvl,AllPub,...,0,0,-1,MnPrv,-1,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,-1,IR1,Lvl,AllPub,...,0,0,-1,-1,-1,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,-1,IR1,HLS,AllPub,...,144,0,-1,-1,-1,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,-1,Reg,Lvl,AllPub,...,0,0,-1,-1,-1,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,-1,Reg,Lvl,AllPub,...,0,0,-1,-1,-1,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,-1,Reg,Lvl,AllPub,...,0,0,-1,-1,-1,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,-1,Reg,Lvl,AllPub,...,0,0,-1,MnPrv,Shed,700,7,2006,WD,Normal


In [267]:
output = pd.DataFrame({'Id': test_data['Id'],
                       'SalePrice' : test_preds})
output.to_csv('submission.csv',index=False)
print('File created')

File created


In [268]:
output.head()

,Id,SalePrice
0,1461,128588.237143
1,1462,155344.228571
2,1463,180664.795714
3,1464,182676.571429
4,1465,199077.294286
